# Поиск токсичных комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Нужно обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.

Необходима модель со значением метрики качества *F1* не меньше 0.75.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
!pip install scikit-learn optuna transformers datasets -U

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import pipeline
from datasets import Dataset, DatasetDict, load_from_disk
from sklearn.model_selection import train_test_split
from sklearn.metrics import get_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
import optuna
import warnings
import joblib
import os

In [3]:
optuna.logging.set_verbosity('ERROR')

In [4]:
SEED = abs(hash('BLOODSHED')) % (2**32)
N_SAMPLES = 5000
BATCH_SIZE = 100

Загружаем данные и выводим общую информацию

In [5]:
use_google_drive = True #@param {type:"boolean"}
if use_google_drive:
    if not os.path.exists('/content/gdrive/MyDrive/'):
        from google.colab import drive
        drive.mount('/content/gdrive')
    path = '/content/gdrive/MyDrive'
else:
    path = '.'

df = pd.read_csv(f'{path}/toxic_comments.csv', index_col=0)
df.info()
df[:5]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
df.duplicated().sum()

0

Проверим баланс классов в данных

In [7]:
def class_balance(targets):
    return (targets
            .value_counts()
            .to_frame('count')
            .assign(fraction=lambda x: x['count'] / targets.shape[0])
            .rename_axis('class'))

class_balance(df['toxic'])

,count,fraction
class,,
0,143106,0.898388
1,16186,0.101612


In [8]:
try:
    ds = load_from_disk(f'{path}/dataset/')
    print('Датасет загружен успешно.')
    print('Можно пропустить предобработку и сразу перейти к обучению.')
except FileNotFoundError:
    print('Датасет на диске не найден.')
    print('Следующие ячейки создадут его из датафрейма автоматически.')

Датасет загружен успешно.
Можно пропустить предобработку и сразу перейти к обучению.


Этот код создаст датасет размером `N_SAMPLES`, разделённый на выборки для обучения, валидации и тестирования в соотношении 60:20:20

In [ ]:
train, valid = train_test_split(df,
                                test_size=int(N_SAMPLES * 0.4),
                                random_state=SEED,
                                stratify=df['toxic'])

n_samples = (N_SAMPLES - valid.shape[0]) // df['toxic'].nunique()
train = (train.groupby('toxic', group_keys=False)
              .apply(lambda x: x.sample(n_samples, random_state=SEED)))

valid, test = train_test_split(valid,
                               test_size=0.5,
                               random_state=SEED,
                               stratify=valid['toxic'])

sets = train, valid, test
names = 'train', 'valid', 'test'
ds = DatasetDict(zip(names, map(Dataset.from_pandas, sets)))

del df, train, valid, test, sets

ds.shape

{'train': (3000, 3), 'valid': (1000, 3), 'test': (1000, 3)}

Посмотрим на баланс классов в получившемся датасете

In [ ]:
for n, s in ds.items():
    print(f'{f" {n.title()} ":=^24}')
    display(class_balance(pd.Series(s['toxic'])))

======== Train =========


,count,fraction
class,,
0,1500,0.5
1,1500,0.5


======== Valid =========


,count,fraction
class,,
0,898,0.898
1,102,0.102


========= Test =========


,count,fraction
class,,
0,899,0.899
1,101,0.101


Создаём pipeline с токенизатором и моделью BERT

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
pipe = pipeline('feature-extraction',
                tokenizer='bert-base-uncased',
                model='bert-base-uncased',
                framework='pt',
                device=device)

In [ ]:
def bert(x):
    features = pipe([x['text']],
                    tokenize_kwargs=dict(
                        padding=True,
                        truncation=True,
                        add_special_tokens=True,
                    ),
                    return_tensors=True)
    x['features'] = features[0][:,0,:]
    return x
ds = ds.map(bert, batched=True, batch_size=BATCH_SIZE)
ds.set_format('numpy')

del pipe

if not os.path.exists(f'{path}/dataset/'):
    !mkdir {path}/dataset
ds.save_to_disk(f'{path}/dataset/')

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Итак, чтобы подготовить данные:
- Проверено отсутствие пропусков и дубликатов
- Для `train` выборки взято сбалансированное количество примеров для каждого класса. Для `valid` и `test` выборок баланс классов не изменён
- Текст преобразован в векторы моделью BERT
- Векторизированные данные разбиты на выборки для обучения, валидации и тестирования

## Обучение

Функция считающая метрику (по умолчанию F1) для модели на валидации или тестровании в зависимости от аргументов

In [28]:
xtrain = ds['train']['features']
ytrain = ds['train']['toxic']
xvalid = ds['valid']['features']
yvalid = ds['valid']['toxic']
xtest = ds['test']['features']
ytest = ds['test']['toxic']

def score(model, test=False, scoring='f1'):
    if test:
        x = np.concatenate((xtrain, xvalid))
        y = np.concatenate((ytrain, yvalid))
        xv, yv = xtest, ytest
    else:
        x, y = xtrain, ytrain
        xv, yv = xvalid, yvalid

    model.fit(x, y)
    pred = model.predict(xv)
    if isinstance(scoring, list):
        return (get_scorer(s)._score_func(yv, pred) for s in scoring)
    return get_scorer(scoring)._score_func(yv, pred)

Создадим датафрейм в котором будем сохранять результаты обучения моделей, чтобы позже легче было их сравнить.

In [10]:
try:
    res = joblib.load(f'{path}/res.pkl')
except FileNotFoundError:
    res = pd.DataFrame(columns=['F1', 'trial'])

Функция оптимизирующая гиперпараметры модели и сохраняющая результаты в датафрейм. В конце выводит гиперпараметры лучшей модели и её F1 на валидации.

In [11]:
def optimize(name, model_fn, n_trials=1, timeout=None):
    if name in res.index:
        value, trial = res.loc[name]
    else:
        sampler = optuna.samplers.TPESampler(seed=SEED)
        study = optuna.create_study(sampler=sampler, direction='maximize')

        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            study.optimize(lambda t: score(model_fn(t)),
                           n_trials=n_trials,
                           timeout=timeout,
                           show_progress_bar=True)

        trial = study.best_trial
        value = study.best_trial.value

        res.loc[name] = value, trial

    print(f'params: {trial.params}')
    print(f'F1: {value:.4f}')

Обучаем модели:

In [12]:
optimize('Logit', lambda _: LogisticRegression(random_state=SEED))

  0%|          | 0/1 [00:00<?, ?it/s]

params: {}
F1: 0.5280


In [13]:
def logit(trial):
    params = {
        'penalty': trial.suggest_categorical('penalty', [None,
                                                         'l1',
                                                         'l2',
                                                         'elasticnet']),
        'C': trial.suggest_float('C', 0.01, 100, log=True),
        'tol': trial.suggest_float('tol', 1e-4, 1, log=True),
    }
    if params['penalty'] == 'elasticnet':
        params['l1_ratio'] = trial.suggest_float('l1_ratio', 0.01, 0.99)
    return LogisticRegression(random_state=SEED,
                              max_iter=100_000,
                              solver='saga',
                              **params)

optimize('Logit Tuned', logit, n_trials=100)

  0%|          | 0/100 [00:00<?, ?it/s]

params: {'penalty': 'l1', 'C': 0.06816464504247255, 'tol': 0.6500051508293069}
F1: 0.5683


In [14]:
optimize('Decision Tree', lambda _: DecisionTreeClassifier(random_state=SEED))

  0%|          | 0/1 [00:00<?, ?it/s]

params: {}
F1: 0.3325


In [15]:
def decision_tree(trial):
    params = {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 30),
    }
    return DecisionTreeClassifier(random_state=SEED, **params)

optimize('Decision Tree Tuned', decision_tree, n_trials=75)

  0%|          | 0/75 [00:00<?, ?it/s]

params: {'criterion': 'gini', 'max_depth': 13, 'min_samples_split': 3, 'min_samples_leaf': 29}
F1: 0.3663


In [16]:
optimize('Random Forest', lambda _: RandomForestClassifier(random_state=SEED))

  0%|          | 0/1 [00:00<?, ?it/s]

params: {}
F1: 0.5031


In [17]:
def random_forest(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 1, 300),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 1, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 30),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 30),
    }
    return RandomForestClassifier(random_state=SEED, **params)

optimize('Random Forest Tuned', random_forest, n_trials=50)

  0%|          | 0/50 [00:00<?, ?it/s]

params: {'n_estimators': 219, 'criterion': 'gini', 'max_depth': 22, 'min_samples_split': 6, 'min_samples_leaf': 6}
F1: 0.5109


In [18]:
optimize('Hist Gradient Boosting',
         lambda _: HistGradientBoostingClassifier(random_state=SEED))

  0%|          | 0/1 [00:00<?, ?it/s]

params: {}
F1: 0.5296


In [19]:
def hist_gradient_boosting(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1),
        'max_iter': trial.suggest_int('max_iter', 1, 300),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 300),
        'max_depth': trial.suggest_int('max_depth', 1, 50),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 50),
    }
    return HistGradientBoostingClassifier(random_state=SEED, **params)

optimize('Hist Gradient Boosting Tuned', hist_gradient_boosting, n_trials=50)

  0%|          | 0/50 [00:00<?, ?it/s]

params: {'learning_rate': 0.24486484858068014, 'max_iter': 161, 'max_leaf_nodes': 132, 'max_depth': 8, 'min_samples_leaf': 50}
F1: 0.5770


Сохраняем и визуализируем результаты:

In [20]:
_ = joblib.dump(res, f'{path}/res.pkl')

In [21]:
res['F1'].to_frame().style.format(precision=4).highlight_max(color='green')

,F1
Logit,0.5280
Logit Tuned,0.5683
Decision Tree,0.3325
Decision Tree Tuned,0.3663
Random Forest,0.5031
Random Forest Tuned,0.5109
Hist Gradient Boosting,0.5296
Hist Gradient Boosting Tuned,0.5770


Классические модели не достигли нужного качества. Попробуем применить нейронную сеть специально обученную для определения токсичных комментариев.

In [23]:
class ToxicBert():
    def __init__(self, device=None):
        if device is None:
            device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        self.pipe = pipeline('text-classification',
                             tokenizer='unitary/toxic-bert',
                             model='unitary/toxic-bert',
                             framework='pt',
                             device=device)

    def fit(self, x, y=None):
        return self

    def predict_proba(self, x):
        res = self.pipe(x,
                        top_k=None,
                        padding=True,
                        truncation=True,
                        add_special_tokens=True)
        proba = np.fromiter((next(c['score']
                                  for c in r
                                  if c['label'] == 'toxic')
                             for r in res),
                            dtype=float)
        return proba

    def predict(self, x):
        proba = self.predict_proba(x)
        return np.rint(proba).astype(int)

In [26]:
ds.set_format()
xtrain = ds['train']['text']
xvalid = ds['valid']['text']
xtest = ds['test']['text']

model = ToxicBert()
value = score(model)
print(f'F1: {value:.4f}')

F1: 0.9510


## Тестирование

In [29]:
f1, acc = score(model, test=True, scoring=['f1', 'accuracy'])
print(f'F1 Test: {f1:.4f}')
print(f'Accuracy: {acc:.4f}')

F1 Test: 0.9239
Accuracy: 0.9850


Отлично! Метрика F1 лучшей модели на тесте почти такая же как на валидации, значит у модели хорошо получилось обобщить данные.

Теперь сравним качество с простыми моделями

In [30]:
for s in [0, 1, 'stratified', 'uniform']:
    if isinstance(s, str):
        p = dict(strategy=s)
        print(f'Dummy {s.title()}')
    else:
        p = dict(strategy='constant', constant=s)
        print(f'Dummy Constant {s}')
    model = DummyClassifier(**p)
    f1, acc = score(model, test=True, scoring=['f1', 'accuracy'])
    print(f'F1: {f1:.4f}')
    print(f'Accuracy: {acc:.4f}')
    print()

Dummy Constant 0
F1: 0.0000
Accuracy: 0.8990

Dummy Constant 1
F1: 0.1835
Accuracy: 0.1010

Dummy Stratified
F1: 0.1720
Accuracy: 0.5860

Dummy Uniform
F1: 0.1463
Accuracy: 0.4980



Итоговое качество намного лучше чем у простых моделей, значит модель адекватна

## Выводы

По результатам исследования:
- Комментарии векторизированы моделью BERT
- Обучены разные модели для классификации комментариев на позитивные и негативные
- Выбрана и протестирована лучшая модель
- Проверена адекватность и качество модели